# Dynamics 365 認証サンプルノートブック
このノートブックは、.envファイルからシークレットを安全に読み込んでDynamics 365に接続するサンプルです。
- シークレットはコードに直接書かず、.envファイルで管理してください。
- .envは.gitignoreで必ず除外してください。

In [ ]:
# 必要なライブラリのインポート
from msal import ConfidentialClientApplication
import requests
import os
from dotenv import load_dotenv

load_dotenv()  # .envファイルから環境変数を読み込む

In [ ]:
# 本番環境の認証情報を環境変数から取得
resource_url = os.getenv('RESOURCE_URL')
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

authority_url = f'https://login.microsoftonline.com/{tenant_id}'
scope = [f'{resource_url}/.default']

app = ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
token = app.acquire_token_for_client(scopes=scope)
if 'access_token' not in token:
    raise RuntimeError(f'Failed to acquire access token: {token.get('error_description', token)}')
access_token = token['access_token']

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json'
}

In [ ]:
# サンドボックス環境の認証情報（必要な場合のみ）
sandbox_resource_url = os.getenv('SANDBOX_RESOURCE_URL')
sandbox_tenant_id = os.getenv('SANDBOX_TENANT_ID') or tenant_id
sandbox_client_id = os.getenv('SANDBOX_CLIENT_ID') or client_id
sandbox_client_secret = os.getenv('SANDBOX_CLIENT_SECRET') or client_secret

sandbox_authority_url = f'https://login.microsoftonline.com/{sandbox_tenant_id}'
sandbox_scope = [f'{sandbox_resource_url}/.default']

sandbox_app = ConfidentialClientApplication(
    sandbox_client_id,
    authority=sandbox_authority_url,
    client_credential=sandbox_client_secret
)
sandbox_token = sandbox_app.acquire_token_for_client(scopes=sandbox_scope)
if 'access_token' not in sandbox_token:
    raise RuntimeError(f'Failed to acquire sandbox access token: {sandbox_token.get('error_description', sandbox_token)}')
sandbox_access_token = sandbox_token['access_token']

sandbox_headers = {
    'Authorization': f'Bearer {sandbox_access_token}',
    'Content-Type': 'application/json',
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json'
}

In [ ]:
# shirakuraindustry環境への接続・認証（トークン取得）
from msal import ConfidentialClientApplication
import requests
import time
import os
from dotenv import load_dotenv

load_dotenv()  # .envファイルから環境変数を読み込む

# 本番用の設定
resource_url = os.getenv('RESOURCE_URL')
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

authority_url = f'https://login.microsoftonline.com/{tenant_id}'
scope = [f'{resource_url}/.default']

app = ConfidentialClientApplication(client_id, authority=authority_url, client_credential=client_secret)
token = app.acquire_token_for_client(scopes=scope)
if "access_token" not in token:
    raise RuntimeError(f"Failed to acquire access token: {token.get('error_description', token)}")
access_token = token['access_token']

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json'
}

print('shirakuraindustry環境への接続・認証が完了しました')

shirakuraindustry環境への接続・認証が完了しました


In [ ]:
# サンドボックス環境への接続・認証（トークン取得）
from msal import ConfidentialClientApplication
import requests
import time
import os
from dotenv import load_dotenv

load_dotenv()  # .envファイルから環境変数を読み込む

# サンドボックス用の設定（必要に応じて変更）
sandbox_resource_url = os.getenv('SANDBOX_RESOURCE_URL') or 'https://org9a8c5649.crm7.dynamics.com/'
sandbox_tenant_id = os.getenv('SANDBOX_TENANT_ID') or os.getenv('TENANT_ID')
sandbox_client_id = os.getenv('SANDBOX_CLIENT_ID') or os.getenv('CLIENT_ID')
sandbox_client_secret = os.getenv('SANDBOX_CLIENT_SECRET') or os.getenv('CLIENT_SECRET')

sandbox_authority_url = f'https://login.microsoftonline.com/{sandbox_tenant_id}'
sandbox_scope = [f'{sandbox_resource_url}/.default']

sandbox_app = ConfidentialClientApplication(
    sandbox_client_id,
    authority=sandbox_authority_url,
    client_credential=sandbox_client_secret
)
sandbox_token = sandbox_app.acquire_token_for_client(scopes=sandbox_scope)
if "access_token" not in sandbox_token:
    raise RuntimeError(f"Failed to acquire sandbox access token: {sandbox_token.get('error_description', sandbox_token)}")
sandbox_access_token = sandbox_token["access_token"]

sandbox_headers = {
    'Authorization': f'Bearer {sandbox_access_token}',
    'Content-Type': 'application/json',
    'OData-MaxVersion': '4.0',
    'OData-Version': '4.0',
    'Accept': 'application/json'
}

print('サンドボックス環境への接続・認証が完了しました')

サンドボックス環境への接続・認証が完了しました


In [18]:
# 在庫調整（msdyn_inventoryadjustments）レコードを作成（倉庫指定あり）
warehouse_id = '5b743789-c329-41ee-89e5-f81b83570131'  # 必要に応じて変更
adj_url = f"{sandbox_resource_url.rstrip('/')}/api/data/v9.2/msdyn_inventoryadjustments"
adj_data = {
    "msdyn_warehouse@odata.bind": f"/msdyn_warehouses({warehouse_id})"
}
adj_res = requests.post(adj_url, headers=sandbox_headers, json=adj_data)
if adj_res.status_code in [200, 201, 204]:
    adj_json = adj_res.json() if adj_res.content else {}
    adj_id = adj_json.get("msdyn_inventoryadjustmentid") or adj_res.headers.get("OData-EntityId", "").split("(")[-1].split(")")[0]
    print(f'在庫調整ID: {adj_id}')
else:
    print(f'在庫調整レコード作成失敗: {adj_res.status_code}')
    print(adj_res.text)

在庫調整ID: 221fc61c-56a6-f011-bbd3-00224862d2f6


In [19]:
# 指定倉庫（warehouse_id）の製品在庫をリスト化
warehouse_id = '5b743789-c329-41ee-89e5-f81b83570131'
productinventory_table = 'msdyn_productinventories'
primary_key = 'msdyn_productinventoryid'
product_field = '_msdyn_product_value'
warehouse_field = '_msdyn_warehouse_value'
qtyonhand_field = 'msdyn_qtyonhand'
uom_field = '_msdyn_unit_value'  # 単位（UoM）の外部キー
base_url = f"{sandbox_resource_url.rstrip('/')}/api/data/v9.2/{productinventory_table}"
# $filterで指定倉庫のみ抽出
filter_query = f"?$select={primary_key},{product_field},{warehouse_field},{qtyonhand_field},{uom_field}&$filter={warehouse_field} eq {warehouse_id!r}"
url = base_url + filter_query
print('Request URL:', url)
all_records = []
while url:
    response = requests.get(url, headers=sandbox_headers)
    if response.status_code == 200:
        data = response.json()
        all_records.extend(data.get('value', []))
        url = data.get('@odata.nextLink')
    else:
        print(f'取得エラー: {response.status_code}')
        print(response.text)
        break
print(f'倉庫 {warehouse_id} の製品在庫件数: {len(all_records)}')
import pandas as pd
df = pd.DataFrame(all_records)
display(df)

Request URL: https://org9a8c5649.crm7.dynamics.com/api/data/v9.2/msdyn_productinventories?$select=msdyn_productinventoryid,_msdyn_product_value,_msdyn_warehouse_value,msdyn_qtyonhand,_msdyn_unit_value&$filter=_msdyn_warehouse_value eq '5b743789-c329-41ee-89e5-f81b83570131'
倉庫 5b743789-c329-41ee-89e5-f81b83570131 の製品在庫件数: 347


,@odata.etag,_msdyn_product_value,msdyn_qtyonhand,_msdyn_warehouse_value,_msdyn_unit_value,msdyn_productinventoryid
0,"W/""210371756""",984d43e2-1b17-ef11-840a-6045bd533858,8.00,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,d60751fc-1e56-ef11-bfe3-000d3acef57b
1,"W/""210372074""",fa26752a-d24b-ee11-be6e-002248ef828a,2.40,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,83bc8f21-2164-ee11-8def-000d3acf4c8b
2,"W/""210372080""",dcf7e754-d34b-ee11-be6e-002248ef828a,2.50,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,051eb144-2164-ee11-8def-000d3acf4c8b
3,"W/""210371486""",95d463bc-90d7-ed11-a7c7-002248627fc5,1.52,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,a1756b8e-badd-ed11-a7c7-00224862078b
4,"W/""144983330""",97f7ffb6-0a60-ed11-9562-002248627fc5,0.00,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,36e63c11-3bd0-ed11-a7c6-002248627ef1
...,...,...,...,...,...,...
342,"W/""210473979""",33456331-8eb6-ed11-b597-002248627fc5,3.00,5b743789-c329-41ee-89e5-f81b83570131,913f0395-4d19-ed11-b83e-000d3acf194f,3005a855-6988-f011-b4cb-6045bd54e2d0
343,"W/""210373148""",a3ee263a-8db6-ed11-b597-002248627fc5,6.00,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,559d6c68-6988-f011-b4cb-6045bd54e2d0
344,"W/""210325701""",0d7ee18c-ecc3-ed11-b597-002248627fc5,0.00,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,583ecd74-6988-f011-b4cb-6045bd54e2d0
345,"W/""210371603""",fefcf29b-c990-ef11-8a6a-6045bd5478c8,6.20,5b743789-c329-41ee-89e5-f81b83570131,8d3f0395-4d19-ed11-b83e-000d3acf194f,2603510c-038a-f011-b4cb-6045bd54e2d0


In [ ]:
# 既存の在庫調整IDに、倉庫内すべての製品在庫を100にする調整明細を追加
from tqdm import tqdm
import re

adj_id = '221fc61c-56a6-f011-bbd3-00224862d2f6'  # 既存の在庫調整ID
product_field = '_msdyn_product_value'
qtyonhand_field = 'msdyn_qtyonhand'
uom_field = '_msdyn_unit_value'

if 'df' not in locals():
    raise RuntimeError('先に倉庫内在庫リスト化セルを実行してください')

all_records = df.to_dict(orient='records')
if not all_records:
    print('レコードがありません')
else:
    for rec in tqdm(all_records, desc='POST中', unit='件'):
        product_id = rec.get(product_field)
        uom_id = rec.get(uom_field)

        # msdyn_qtyonhandの値を100にする調整
        diff = 100 - (rec.get(qtyonhand_field) or 0)
        if product_id and diff != 0:
            adjprod_url = f"{sandbox_resource_url.rstrip('/')}/api/data/v9.2/msdyn_inventoryadjustmentproducts"
            adjprod_data = {
                "msdyn_inventoryadjustment@odata.bind": f"/msdyn_inventoryadjustments({adj_id})",
                "msdyn_product@odata.bind": f"/products({product_id})",
                "msdyn_quantity": diff
            }
            if uom_id:
                adjprod_data["msdyn_unit@odata.bind"] = f"/uoms({uom_id})"
            adjprod_res = requests.post(adjprod_url, headers=sandbox_headers, json=adjprod_data)
            if adjprod_res.status_code in [200, 201, 204]:
                tqdm.write(f"✅ 調整明細作成: Product={product_id}, 差分={diff}, Unit={uom_id}, Warehouse={warehouse_id}")
            else:
                tqdm.write(f"❌ 調整明細作成失敗: Product={product_id} - {adjprod_res.status_code}")
                tqdm.write(adjprod_res.text)
            time.sleep(0.05)  # API負荷軽減
        else:
            tqdm.write(f"⏭ スキップ: Product={product_id}, Diff={diff}")

In [73]:
# 認証・ヘッダーは既存セルを利用
project_table = 'cr455_projects'
target_title = 'S2507073'
url = f"{resource_url}/api/data/v9.2/{project_table}?$filter=cr455_title eq '{target_title}'"
response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    if 'value' in data and len(data['value']) > 0:
        record = data['value'][0]
        print('取得したレコード:')
        print(record)
    else:
        print('該当レコードが見つかりません')
else:
    print(f'エラー: {response.status_code}')
    print(response.text)

取得したレコード:
{'@odata.etag': 'W/"217969404"', 'cr455_construction_workprogress': 0, 'cr455_uriage': None, '_cr455_representativeoftheprimarycontractor_value': None, 'cr455_uriage_base': None, 'cr455_amount': 0.0, 'cr455_grn_siteform': 2, '_cr455_primarycontractor_value': None, '_cr455_s_sales_contact_value': '95e72436-6067-ef11-a670-000d3acf7f2d', 'cr455_scaffold_day': '2025-10-01T15:00:00Z', '_transactioncurrencyid_value': '5888cedb-4416-ed11-b83d-000d3acee53f', 'cr455_amount_base': 0.0, 'cr455_title': 'S2507073', 'cr455_constructnumbers': '8412507481000', 'cr455_period_date': None, 'cr455_orderdate': '2025-09-10T00:00:00Z', 'cr455_area': 9.94, 'cr455_quoteamount_base': 345433.0, '_cr455_project_account_value': '83244798-4619-ed11-b83e-000d3acf194f', 'cr455_schedule_startday': '2025-10-06T15:00:00Z', 'createdon': '2025-07-07T02:04:04Z', 'statecode': 0, 'cr455_processing': False, '_cr455_project_building_product_name_value': '4929d9c9-da64-ef11-a670-000d3acf7f2d', 'cr455_undertaked': Fals

In [85]:
import xml.etree.ElementTree as ET

xml_path = 'CMT Files/cr455_project_data.xml'  # 元ファイル
output_path = 'CMT Files/data.xml'  # 出力ファイル
target_value = 'S2509090'  # 残したい値

tree = ET.parse(xml_path)
root = tree.getroot()

# entityのrecordsノード取得
entity = root.find(".//entity[@name='cr455_project']")
records = entity.find('records') if entity is not None else None

if records is not None:
    for record in list(records):
        # record内のtargetフィールドを探す
        title_field = record.find("field[@name='cr455_title']")
        if title_field is None or title_field.get('value') != target_value:
            records.remove(record)

    # 新しいファイルに保存
    tree.write(output_path, encoding='utf-8', xml_declaration=True)
    print(f"target_valueが{target_value} のみを {output_path} に保存しました")
else:
    print('recordsが見つかりません')

cr455_title=S2509090 のみを CMT Files/data.xml に保存しました


In [95]:
import xml.etree.ElementTree as ET

xml_path = 'CMT Files/cr455_constructionmaterial_data.xml'  # 元ファイル
output_path = 'CMT Files/data.xml'  # 出力ファイル
target_value = '3de894a7-198d-f011-b4cb-002248ef5001'  # 残したい値

tree = ET.parse(xml_path)
root = tree.getroot()

# entityのrecordsノード取得
entity = root.find(".//entity[@name='cr455_constructionmaterial']")
records = entity.find('records') if entity is not None else None

if records is not None:
    for record in list(records):
        # record内のtargetフィールドを探す
        target_field = record.find("field[@name='cr455_project']")
        if target_field is None or target_field.get('value') != target_value:
            records.remove(record)

    # 新しいファイルに保存
    tree.write(output_path, encoding='utf-8', xml_declaration=True)
    print(f"target_valueが{target_value} のみを {output_path} に保存しました")
else:
    print('recordsが見つかりません')

target_valueが3de894a7-198d-f011-b4cb-002248ef5001 のみを CMT Files/data.xml に保存しました


In [92]:
records

<Element 'records' at 0x000001F1FE0CF740>

In [5]:
# Azure Blob StorageのCDMデータをテーブル形式で読み込むサンプル
from azure.storage.blob import BlobServiceClient
import pandas as pd
import json
import io
from urllib.parse import urlparse, unquote

# Azure Blob Storageの情報
account_url = "https://4x7u1zn3zy3hs612.blob.core.windows.net"
container_name = "power-platform-dataflows"
manifest_path = "environments/shirakuraindustry/D365 Data Imigration/model.json"
sas_token = "?sv=2025-07-05&ss=btqf&srt=sco&spr=https&st=2025-10-12T05%3A57%3A03Z&se=2025-10-13T05%3A57%3A03Z&sp=rl&sig=toLEaOucKJEZJeO8eH5fFlyGRyuvYwInrABvMDMqB1Q%3D"

# BlobServiceClientの作成
blob_service_client = BlobServiceClient(account_url=account_url, credential=sas_token)
container_client = blob_service_client.get_container_client(container_name)

# manifest.jsonのダウンロード
blob_client = container_client.get_blob_client(manifest_path)
manifest_data = blob_client.download_blob().readall()
manifest = json.loads(manifest_data)

# データファイルのパスを取得（例: 最初のエンティティの最初のpartition）
entity = manifest["entities"][0]
data_path = entity["partitions"][0]["location"]

# data_pathが絶対URLの場合、blobのパス部分だけを抽出
parsed = urlparse(data_path)
blob_path = unquote(parsed.path)
if blob_path.startswith(f'/{container_name}/'):
    blob_path = blob_path[len(f'/{container_name}/'):]
elif blob_path.startswith('/'):
    blob_path = blob_path[1:]

data_blob_client = container_client.get_blob_client(blob_path)

# ヘッダ情報の取得（CDMのattributesから列名リストを作成）
columns = [attr['name'] for attr in entity.get('attributes', [])]

# ファイル形式で処理を分岐
if blob_path.lower().endswith('.parquet'):
    data_bytes = data_blob_client.download_blob().readall()
    df = pd.read_parquet(io.BytesIO(data_bytes))
    if columns and len(columns) == len(df.columns):
        df.columns = columns
elif blob_path.lower().endswith('.csv') or '.csv@' in blob_path:
    data_bytes = data_blob_client.download_blob().readall()
    if columns:
        df = pd.read_csv(io.BytesIO(data_bytes), names=columns, header=None)
    else:
        df = pd.read_csv(io.BytesIO(data_bytes))
else:
    raise ValueError(f"未対応のファイル形式: {blob_path}")

print(df.head())

                              accountid  accountcategorycode  \
0  7628C8B4-4219-ED11-B83E-000D3ACEDC8D                  NaN   
1  4DEC76A5-4219-ED11-B83E-000D3ACF194F                  NaN   
2  386BC425-27A7-F011-BBD3-002248EF5D56                  NaN   

   customersizecode  preferredcontactmethodcode  customertypecode  \
0                 1                           1               5.0   
1                 1                           1              11.0   
2                 1                           1               NaN   

   accountratingcode  industrycode  territorycode  accountclassificationcode  \
0                  1            53              1                          1   
1                  1            56              1                          1   
2                  1            56              1                          1   

   businesstypecode  ... msft_datastate  msft_datastatename  \
0                 1  ...            NaN                 NaN   
1                 1